# Coleridgh Initiabive 日本語EDA
## お役に立ちましたらupvoteお願いいたします !

# 1. 要約
## 簡単に言えば、学術論文の全文章内容から、dataset_labelという何について書かれているかの分類分けをするコンペだと思います。

In [ ]:
import numpy as np 
import pandas as pd
import os
import random
import json
from tqdm import tqdm

In [ ]:
import re
import glob
from textblob import TextBlob
from functools import partial
import string
import nltk
import spacy
from nltk.probability import FreqDist


In [ ]:
RANDOM_SEED = 42
def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
seed_everything()

# 2. データの中身確認

In [ ]:
train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
train.head(3)

In [ ]:
for col in train.columns:
    print(col + ":" + str(len(train[col].unique())))

print("all rows = " + str(len(train)))

* Id　14316個 : 学術論文のidです。trainフォルダには、このid + ".json" ファイルがあります。このjsonファイルが論文の全文章になります。


* pub_title 14271個 : 学術論文の出版物のタイトルです。


* dataset_title 45個 : パブリケーション内で言及されているデータセットのタイトル。(※　製作者が命名したものらしいです。)


* dataset_label 130個 : これを予測します。データセットを示すテキストの一部。（※　論文筆者が使っている名前。省略などあるため、dataset_titleより数が多いらしいです。)

                                                                                        ※ 印はコメントいただきました。ありがとうございます！
                                                                                        
                                                                                        

* cleaned_label 130個 : evaluation項目にあるように、dataset_labelを小文字とかきれいに整形したものです。submissionはこの形にする必要があります(関数通すだけです)

* 全部の行は19661個あるのに、たいしてそれぞれ重複しているものがある。（3.2で考察します)

In [ ]:
sample = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
sample

Id : testフォルダにこのid + ".json"ファイルがあります。このjsonファイルが論文の全文章になります。
PreditionString : 後程説明しますが、dataset_labelをここに記載します。複数あると思う場合は、"|"で連結します。

# 3 trainデータの中身確認

In [ ]:
train.head(3)

## 3.1 論文を見る

In [ ]:
train_path = "../input/coleridgeinitiative-show-us-the-data/train"
test_path = "../input/coleridgeinitiative-show-us-the-data/test"

In [ ]:
all_train_path = [os.path.join(train_path,s) + ".json" for s in train["Id"]]
all_test_path = [os.path.join(test_path,s) + ".json" for s in sample["Id"]]


## 3.1.1 まずは１つだけ

In [ ]:
train.head(3)

In [ ]:
json_path = all_train_path[0]
json_path

In [ ]:
with open(json_path, 'r') as f:
        json_decode = json.load(f)

In [ ]:
json_decode[:3] # すごく長いので、表示は最初から3つに。

## 細かく見るとsection titleとtextが交互に辞書型になっているように見える.pandasに食わせてみる

In [ ]:
jsontest = pd.DataFrame(json_decode)
jsontest

### jsonファイルは、section_titleとその中身がtextで書かれていることがわかる。学術論文を想像すればイメージつきやすいですね。

## タイトルとテキストを全文くっつけます。(textだけくっつけたいとかは、以下を編集してください)

In [ ]:
texts = ""

for a in jsontest.values:
    texts += a[0] +" "+ a[1] + " "

In [ ]:
texts[:300]

In [ ]:
json_path = all_train_path[0]
with open(json_path, 'r') as f:
        json_decode = json.load(f)

jsontest = pd.DataFrame(json_decode)

texts = ""

for a in jsontest.values:
    texts += a[0] +" "+ a[1] +" "

## train-data全部に対して、それを行う。(↑の3.1.1というところからここまでをひとくくりにしてfor文で回すだけ)

In [ ]:
%%time

alltexts = []

for json_path in tqdm(all_train_path):

    with open(json_path, 'r') as f:
            json_decode = json.load(f)
    jsontest = pd.DataFrame(json_decode)

    texts = ""

    for a in jsontest.values:
        texts += a[0] +" "+ a[1] + " "
        
    alltexts.append(texts)

In [ ]:
train["text"] = alltexts
train

# 3.2 考察

## 3.2.1 同じid列について。2の結果から、全部で19661行あるのに、idのuniqueは、14316個。つまり、同じ論文でも行を分けて、複数存在している。

In [ ]:
train.head(3)

In [ ]:
print(len(train["Id"].unique()))
print(len(train))

In [ ]:
Idgroup = train.groupby("Id")["dataset_label"].count().reset_index()
Idgroup.columns = ["Id","count"]
Idgroup = Idgroup.sort_values("count").reset_index(drop=True)
Idgroup

## 一番多いIdを見てみる

In [ ]:
mostId = train[train["Id"] == Idgroup["Id"].iloc[-1]]
mostId.head()

#### 同じ論文でも異なるdataset_title,dataset_labelを持っていることがわかる。

### submissionのPredictionStringには、複数のdataset_labelを持つときは、"|"でつなげるとあるので、この場合お試しで作成してみる。

In [ ]:
mostIdlist = mostId["cleaned_label"].to_list()
mostIdlist

In [ ]:
mostIdlist = ("|").join(mostIdlist)
mostIdlist

#### この場合、少し長いが、このようにPredictionStringに記載してsubmitする(後程お試しでsubmitできる形にしています)

In [ ]:
sample

#### 少し冗長になってしまいそうなので、割愛しますが、publicationのtitleの重複なども同様です。同じpublicationでも複数論文があったり、同じ論文の行が何行かあったりします。

# 4.検証

### 4.1 仮説 : 全文章の中に、dataset label, cleaned dataset labelで出てくるワードが含まれていれば、
### そのワードをsubmissionしてあげればよいのでは ?
### それを検証するために、trainデータのこれらの項目が全文章に入っているかの検証をする





#### テストデータには、この中に含まれていないものもあるみたいです。このやり方が正しいわけではなく、検証です。
#### でも、Shopeeコンペみたいに最後はこれとマージするのかなとは思います。

In [ ]:
train.head(3)

## v6 : dataset_labelは、以下の関数を通したらcleaned_labelになるようにevaluationで定義されているが・・・

In [ ]:
# 最後はこの形に
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
check = []
for a in range(len(train)):
    
    if clean_text(train["dataset_label"].iloc[a]) == train["cleaned_label"].iloc[a]:
        check.append(1)
    else:
        check.append(0)

In [ ]:
np.sum(check)/len(train)

#### 1にならない・・・確認してみる

In [ ]:
train["check"] = check
checkdf = train[train["check"]==0]
checkdf.head(3)

In [ ]:
checkdf["dataset_label"].unique()

In [ ]:
checkdf["cleaned_label"].unique()

In [ ]:
clean_text(checkdf["dataset_label"].iloc[0])

#### 最後のスペースが大本は消えていない。

In [ ]:
cleanlabel = []
for a in tqdm(train["cleaned_label"]):
    if a[-1] == " ":
        cleanlabel.append(a[:-1])
    else:
        cleanlabel.append(a)

In [ ]:
check = []
for a in range(len(train)):
    
    if clean_text(train["dataset_label"].iloc[a]) == cleanlabel[a]:
        check.append(1)
    else:
        check.append(0)

In [ ]:
np.sum(check)/len(train)

#### きちんと1になったので、置換します。

In [ ]:
train["cleaned_label"]=cleanlabel

#### ↓は以下を踏まえて。

In [ ]:
train.head(3)

### train["text"]をきれいに

In [ ]:
train["text"] = [clean_text(s) for s in tqdm(train["text"])]
    

### dataset_labelのuniqueのリスト化

In [ ]:
dslabel = [clean_text(s) for s in train["dataset_label"].unique()]

In [ ]:
len(dslabel)

In [ ]:
labeljudge = []
all_labels = []
label_len = []

for a in tqdm(train["text"]):
    labels = []
    for b in dslabel:
        if b in a:
            labels.append(clean_text(b))
            break
    if len(labels)==0:
        labeljudge.append(0)
    else:
        labeljudge.append(1)
    
    #all_labels.append("|".join(labels))
    #label_len.append(len(labels))

In [ ]:
np.sum(labeljudge)/len(train)

#### 100%一致していることを確認。

# 4.2 testデータと提出

#### 上記結果から、テストデータで、exist_labelを含んでいたらdataset_labelとしてcleaningして提出する。※複数該当するときは"|"で繋げるルール。

In [ ]:
sample

## 4.2.1 論文を読み込む

In [ ]:
%%time

alltexts = []

for json_path in tqdm(all_test_path):

    with open(json_path, 'r') as f:
            json_decode = json.load(f)
    jsontest = pd.DataFrame(json_decode)

    texts = ""

    for a in jsontest.values:
        texts += a[0] + " " + a[1] + " "
        
    alltexts.append(texts)

In [ ]:
sample["text"] = alltexts

In [ ]:
sample

## 4.2.2 論文をきれいにする

In [ ]:
sample["text"] = [clean_text(s) for s in tqdm(sample["text"])]

## 4.2.3 dslabelにワードがあれば、その言葉を抜いて結合する

#### v11 少しだけスコアが上がるので、dataset_titleもdslabelに追加します。

In [ ]:
print(len(dslabel))
dstitle = [clean_text(s) for s in train["dataset_title"].unique()]
dslabel = set(dslabel + dstitle) # setを使うことで重複を削除してくれます。
len(dslabel)

In [ ]:
labeljudge = []
all_labels = []
label_len = []

for a in tqdm(sample["text"]):
    labels = []
    for b in dslabel:
        if b in a:
            labels.append(clean_text(b))
            
    if len(labels)==0:
        labeljudge.append(0)
    else:
        labeljudge.append(1)
    
    all_labels.append("|".join(labels))
    label_len.append(len(labels))

In [ ]:
sample["PredictionString"] = all_labels
sample

In [ ]:
sample["PredictionString"].iloc[2]

In [ ]:
sample = sample[["Id","PredictionString"]]

In [ ]:
sample.to_csv("submission.csv",index=False)

In [ ]:
sample

## ここまで見ていただいてありがとうございます。
## 少しでもお役に立ちましたらupvote/followしてもらえると嬉しいです !